# 1) Introduction/Business Problem

Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.

    Problem Description: The stakeholder wants to open an affordable Vegan/Vegetarian restaurant for an underserved populations. The most desirable location would be one that is poplous with other Vegan/Vegetarian restaurants, yet where residents have a low average income. The study will be used to display the neighborhoods with the highest Vegan/Vegetarian venues, lowest income, and least restaurants that will offer anyone seeking to open a new business the opportunity to leverage competitive data to make their decision.

# 2) Data

Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.

    The data being used in this study will be sourced from the Toronto City Government's Open Resources website (https://www.toronto.ca/city-government/data-research-maps/open-data/learning-resources/Toronto's) and the 2016 Toronto Census. Neighborhood Population, Average Income per Neighborhood with Foursquare API to collect competitors on the same neighborhoods.

In [50]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [51]:
from pandas import DataFrame
from geopy.geocoders import Nominatim
import folium

# Methodology

# Toronto Boroughs and Neighborhoods

In [52]:
df = pd.read_pickle('Toronto')

In [13]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# First, Scarborough and Venues

In [14]:
scarborough_data= df[df['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="ga_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough, CA are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough, CA are 43.773077, -79.257774.


In [16]:
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

In [17]:
CLIENT_ID = "BWJHK44KKVHGIOELR54W55YSVCZOTFPTP1C4IHZIFAUDXRZN"
CLIENT_SECRET = "ZMJ1PWTLSVYGZIMF5X0BV3M4MAXK3FPUJT41CUBOLBRCTSTR"
VERSION = "20200712"
LIMIT = 30
print("Your credentials")
print("CLIENT_ID: " + CLIENT_ID)
print("CLIENT_SECRET: " + CLIENT_SECRET)

Your credentials
CLIENT_ID: BWJHK44KKVHGIOELR54W55YSVCZOTFPTP1C4IHZIFAUDXRZN
CLIENT_SECRET: ZMJ1PWTLSVYGZIMF5X0BV3M4MAXK3FPUJT41CUBOLBRCTSTR


In [18]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [19]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude'])

Found 88 venues in 17 neighborhoods.


In [20]:
print(scarborough_venues.shape)
scarborough_venues.head()

(88, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank


In [30]:
scarborough_veg = scarborough_venues[scarborough_venues['Venue Category'] == 'Vegetarian / Vegan Restaurant']
scarborough_veg.head(9)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


# Secondly, Downtown Toronto

In [22]:
metro_data= df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
metro_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [38]:
metro_venues = getNearbyVenues(names=metro_data['Neighborhood'],
                                   latitudes=metro_data['Latitude'],
                                   longitudes=metro_data['Longitude'])

metro_veg = metro_venues[metro_venues['Venue Category'] == 'Vegetarian / Vegan Restaurant']
metro_veg.head(9)

Found 1228 venues in 19 neighborhoods.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
76,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,The Green Beet Cafe,43.662096,-79.394153,Vegetarian / Vegan Restaurant
204,St. James Town,43.651494,-79.375418,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
266,Berczy Park,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
343,Central Bay Street,43.657952,-79.387383,Vegetarian Haven,43.656016,-79.392758,Vegetarian / Vegan Restaurant
410,"Richmond, Adelaide, King",43.650571,-79.384568,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant
562,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,Kupfert & Kim,43.641179,-79.378144,Vegetarian / Vegan Restaurant
647,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant
747,"Commerce Court, Victoria Hotel",43.648198,-79.379817,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
787,"Commerce Court, Victoria Hotel",43.648198,-79.379817,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant


# 3. Borough: Etobicoke 

In [31]:
etobicoke_data= df[df['Borough'] == 'Etobicoke'].reset_index(drop=True)
etobicoke_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
1,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724
2,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201
3,M9P,Etobicoke,Westmount,43.696319,-79.532242
4,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724


In [36]:
etobi_venues = getNearbyVenues(names=etobicoke_data['Neighborhood'],
                                   latitudes=etobicoke_data['Latitude'],
                                   longitudes=etobicoke_data['Longitude'])

etobi_veg = etobi_venues[etobi_venues['Venue Category'] == 'Vegetarian / Vegan Restaurant']
etobi_veg.head(9)

Found 72 venues in 12 neighborhoods.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


# Borough North York 

In [37]:
nyork_data= df[df['Borough'] == 'North York'].reset_index(drop=True)
nyork_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [39]:
nyork_venues = getNearbyVenues(names=nyork_data['Neighborhood'],
                                   latitudes=nyork_data['Latitude'],
                                   longitudes=nyork_data['Longitude'])

nyork_veg = nyork_venues[nyork_venues['Venue Category'] == 'Vegetarian / Vegan Restaurant']
nyork_veg.head(9)

Found 239 venues in 24 neighborhoods.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


# Borough East York

In [40]:
eyork_data= df[df['Borough'] == 'East York'].reset_index(drop=True)
eyork_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
1,M4C,East York,Woodbine Heights,43.695344,-79.318389
2,M4G,East York,Leaside,43.709060,-79.363452
3,M4H,East York,Thorncliffe Park,43.705369,-79.349372
4,M4J,East York,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106


In [41]:
eyork_venues = getNearbyVenues(names=eyork_data['Neighborhood'],
                                   latitudes=eyork_data['Latitude'],
                                   longitudes=eyork_data['Longitude'])

eyork_veg = eyork_venues[eyork_venues['Venue Category'] == 'Vegetarian / Vegan Restaurant']
eyork_veg.head(9)

Found 77 venues in 5 neighborhoods.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


# Borough: York

In [43]:
york_data= df[df['Borough'] == 'York'].reset_index(drop=True)
york_data.head()

york_venues = getNearbyVenues(names=york_data['Neighborhood'],
                                   latitudes=york_data['Latitude'],
                                   longitudes=york_data['Longitude'])

york_veg = york_venues[york_venues['Venue Category'] == 'Vegetarian / Vegan Restaurant']
york_veg.head(9)

Found 16 venues in 5 neighborhoods.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [46]:
etor_data= df[df['Borough'] == 'East Toronto'].reset_index(drop=True)
etor_data.head()

etor_venues = getNearbyVenues(names=etor_data['Neighborhood'],
                                   latitudes=etor_data['Latitude'],
                                   longitudes=etor_data['Longitude'])

etor_veg = etor_venues[etor_venues['Venue Category'] == 'Vegetarian / Vegan Restaurant']
etor_veg.head(9)

Found 122 venues in 5 neighborhoods.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


# Results/Conclusion

In conclusion of our Foursquare data, Metropolitan or Downtown Toronto is the only Borough that is populated with Vegan/Vegetarian Venues and would therefore offer the most competitive enviroment for a contractor seeking to build an affordable Vegan/Vegetarian Restaurant.

However, if the contractor is seeking to serve an underserved community, the best population would be the borough that is the furthest distance away from Downtown Toronto because they are therefore, furthest away from Vegan/Vegetarian Restaurants.